In [ ]:
import json
import os
import numpy as np
import pandas as pd
import matplotlib.image as mpimg
from matplotlib import pyplot as plt

In [ ]:
collection_path = 'lake_collection'
json_path = 'json'

In [ ]:
with open(os.path.join(json_path, 'lakes_infos.json'), 'r') as f:
    lakes_infos = json.load(f)

with open(os.path.join(json_path, 'binary_images.json'), 'r') as f:
    binary_images = json.load(f)

with open(os.path.join(json_path, 'binary_images_orig.json'), 'r') as f:
    binary_images_orig = json.load(f)

with open(os.path.join(json_path, 'distances_images.json'), 'r') as f:
    distances_images = json.load(f)

with open(os.path.join(json_path, 'distances_images_orig.json'), 'r') as f:
    distances_images_orig = json.load(f)

In [ ]:
def area(bin_image):
    return np.array(bin_image).sum()

In [ ]:
areas = [area(bin_image) for bin_image in binary_images_orig.values()]

In [ ]:
pd.Series(areas).hist(bins=100)

In [ ]:
areas_by_id = {lake_id: area(bin_image) for lake_id, bin_image in binary_images_orig.items()}

In [ ]:
lakes_df = pd.DataFrame(lakes_infos).transpose()[['angle', 'name']]
lakes_df['area'] = pd.Series(areas_by_id)
lakes_df['angle'] = lakes_df['angle'].astype(float)

In [ ]:
quantizer_angle = 20
quantizer_area = 15

In [ ]:
lakes_df.head()

In [ ]:
lakes_df['area_quantile'] = pd.qcut(lakes_df['area'], quantizer_area, labels=range(quantizer_area))

In [ ]:
lakes_df['angle_quantile'] = pd.qcut(lakes_df['angle'], quantizer_angle, labels=range(quantizer_angle))

In [ ]:
def select_one(lakes_df, area_q, angle_q):
    selected_lakes = lakes_df[(lakes_df['area_quantile'] == area_q) & (lakes_df['angle_quantile'] == angle_q)]
    
    if len(selected_lakes) == 0:
        return None
    
    if not all(selected_lakes['name'].isnull()):
        selected_lakes = selected_lakes[selected_lakes['name'].notnull()]
        
    return selected_lakes.sample(1).index.values[0]

In [ ]:
lakes_df['angle'].hist(bins=100)

In [ ]:
id_matrix = np.array(
    [
        [select_one(lakes_df, q_area, q_angle) for q_angle in range(quantizer_angle)]
            for q_area in range(quantizer_area)
    ]
)

In [ ]:
scale = 3

inches_A2 = (23.39 * scale, 16.53 * scale) 

fig, axes = plt.subplots(quantizer_area, quantizer_angle, figsize=inches_A2)

for q_angle in range(quantizer_angle):
    for q_area in range(quantizer_area):
        lake_id = id_matrix[q_area][q_angle]
        
        ax = axes[q_area, q_angle]
        ax.axis('off')
        if lake_id is not None:
            ax.imshow(np.flip(np.transpose(distances_images_orig[lake_id]), axis=0))

        
fig.savefig('poster1.png', bbox_inches='tight')

In [ ]:
fig, axes = plt.subplots(quantizer_area, quantizer_angle, figsize=inches_A2)

for q_angle in range(quantizer_angle):
    for q_area in range(quantizer_area):      
        lake_id = id_matrix[q_area][q_angle]
        
        ax = axes[q_area, q_angle]
        ax.axis('off')
        
        if lake_id is not None:
            im_path = os.path.join(collection_path, lake_id + '_raw.png')
            image = mpimg.imread(im_path)
            ax.imshow(image)       
        
fig.savefig('poster2.png', bbox_inches='tight')

In [ ]:
lakes_df[lakes_df['name'] == 'Lac Léman']

In [ ]:
leman_id = 'relation_332617'

In [ ]:
plt.imshow(np.flip(np.transpose(distances_images_orig[leman_id]), axis=0))

In [ ]:
plt.imshow(distances_images[leman_id])

In [ ]:
plt.imshow(np.flip(np.transpose(binary_images_orig[leman_id]), axis=0))

In [ ]:
image = mpimg.imread('leman.png')

In [ ]:
image = [[1 - rgb[0] for rgb in row] for row in image]

In [ ]:
plt.imshow(image)

In [ ]:
from skimage.morphology import medial_axis

skeleton, distance = medial_axis(np.array(image), return_distance=True)

In [ ]:
plt.imshow(distance)

In [ ]:
distances_images['leman_hand'] = distance.tolist()

In [ ]:
with open(os.path.join('json', 'distances_images_hand.json'), 'w') as f:
    f.write(json.dumps(distances_images))